# Fase 06 — Packaging (versión simplificada)

Esta versión refleja el diseño final simplificado:

- Cada variante F05 aporta **un único modelo oficial**.
- Se copian directamente los **datasets F04** (ya etiquetados).
- No se construye replay desde F02.
- No se seleccionan modelos manualmente.

El objetivo es componer un paquete sellado y reproducible.

In [ ]:
PHASE = "06_packaging"
VARIANT = "v001"  # Cambiar según la variante

## Bootstrap del proyecto

In [ ]:
import sys
from pathlib import Path
import json
from datetime import datetime, timezone
from time import perf_counter
import shutil
import yaml
import pyarrow.parquet as pq

SCRIPT_PATH = Path.cwd().resolve()
ROOT = SCRIPT_PATH
for _ in range(10):
    if (ROOT / "mlops4ofp").exists():
        break
    ROOT = ROOT.parent
else:
    raise RuntimeError("No se pudo localizar project root")

sys.path.insert(0, str(ROOT))
print("Project root:", ROOT)


## Cargar parámetros F06

In [ ]:
from mlops4ofp.tools.params_manager import ParamsManager
from mlops4ofp.tools.run_context import assemble_run_context
from mlops4ofp.tools.artifacts import get_git_hash

pm = ParamsManager(PHASE, ROOT)
pm.set_current(VARIANT)
variant_root = pm.current_variant_dir()

with open(variant_root / "params.yaml", "r", encoding="utf-8") as f:
    params = yaml.safe_load(f)

parent_variants_f05 = params["parent_variants_f05"]
print("Parents F05:", parent_variants_f05)


## Resolver linaje y validar régimen temporal

In [ ]:
lineage = {"f05": set(parent_variants_f05), "f04": set(), "f03": set()}
regimes = set()

for v05 in parent_variants_f05:
    f05_params = yaml.safe_load(
        (ROOT / "executions" / "05_modeling" / v05 / "params.yaml").read_text()
    )
    v04 = f05_params["parent_variant"]
    lineage["f04"].add(v04)

for v04 in lineage["f04"]:
    f04_params = yaml.safe_load(
        (ROOT / "executions" / "04_targetengineering" / v04 / "params.yaml").read_text()
    )
    v03 = f04_params["parent_variant"]
    lineage["f03"].add(v03)

for v03 in lineage["f03"]:
    f03_params = yaml.safe_load(
        (ROOT / "executions" / "03_preparewindowsds" / v03 / "params.yaml").read_text()
    )
    regime = (
        f03_params["temporal"]["Tu"],
        f03_params["temporal"]["OW"],
        f03_params["temporal"]["PW"],
    )
    regimes.add(regime)

if len(regimes) != 1:
    raise RuntimeError("Régimen temporal inconsistente entre F05")

print("Régimen común:", list(regimes)[0])


## Copiar datasets F04

In [ ]:
datasets_dir = variant_root / "datasets"
datasets_dir.mkdir(exist_ok=True)

dataset_paths = []

for v04 in lineage["f04"]:
    src = ROOT / "executions" / "04_targetengineering" / v04 / "04_targetengineering_dataset.parquet"
    dst = datasets_dir / f"{v04}__dataset.parquet"
    shutil.copyfile(src, dst)
    dataset_paths.append(str(dst))

print("Datasets copiados:", dataset_paths)


## Copiar modelos oficiales de cada F05

In [ ]:
models_dir = variant_root / "models"
models_dir.mkdir(exist_ok=True)

selected_models = []

for v05 in parent_variants_f05:
    model_root = ROOT / "executions" / "05_modeling" / v05 / "models"
    model_dirs = [d for d in model_root.iterdir() if d.is_dir()]

    if len(model_dirs) != 1:
        raise RuntimeError(f"F05 {v05} debe contener exactamente un modelo")

    src = model_dirs[0]
    dst = models_dir / f"{v05}__{src.name}"

    if dst.exists():
        shutil.rmtree(dst)

    shutil.copytree(src, dst)

    selected_models.append({"source_f05": v05, "model_id": src.name})

print("Modelos copiados:", selected_models)


## Generar metadata F06

In [ ]:
metadata = {
    "phase": PHASE,
    "variant": VARIANT,
    "git_commit": get_git_hash(),
    "created_at": datetime.now(timezone.utc).isoformat(),
    "temporal": list(regimes)[0],
    "lineage": {k: sorted(v) for k, v in lineage.items()},
    "models": selected_models,
    "datasets": dataset_paths,
}

metadata_path = variant_root / f"{PHASE}_metadata.json"
metadata_path.write_text(json.dumps(metadata, indent=2), encoding="utf-8")

metadata
